In [1]:
import json
import os

def generate_video_paths(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    video_paths = []

    for item in data:
        gloss_list = item['gloss']
        for video in item['videos']:
            if 'video_id' in video:
                video_id = video['video_id']
                for gloss in gloss_list:
                    path = os.path.join('sam_videos', f"{gloss}_{video_id}.mp4")
                    video_paths.append(path)

    return video_paths

# Usage
json_file = 'combined_data.json'
paths = generate_video_paths(json_file)

# Print the paths
for path in paths:
    print(path)

sam_videos/welcome_62837.mp4
sam_videos/basic_62837.mp4
sam_videos/series_62837.mp4
sam_videos/welcome_65143.mp4
sam_videos/basic_65143.mp4
sam_videos/series_65143.mp4
sam_videos/video_26768.mp4
sam_videos/have_26768.mp4
sam_videos/real_26768.mp4
sam_videos/english_26768.mp4
sam_videos/for_26768.mp4
sam_videos/low_26768.mp4
sam_videos/level_26768.mp4
sam_videos/begin_26768.mp4
sam_videos/video_46314.mp4
sam_videos/have_46314.mp4
sam_videos/real_46314.mp4
sam_videos/english_46314.mp4
sam_videos/for_46314.mp4
sam_videos/low_46314.mp4
sam_videos/level_46314.mp4
sam_videos/begin_46314.mp4
sam_videos/video_65634.mp4
sam_videos/have_65634.mp4
sam_videos/real_65634.mp4
sam_videos/english_65634.mp4
sam_videos/for_65634.mp4
sam_videos/low_65634.mp4
sam_videos/level_65634.mp4
sam_videos/begin_65634.mp4
sam_videos/video_23000.mp4
sam_videos/have_23000.mp4
sam_videos/real_23000.mp4
sam_videos/english_23000.mp4
sam_videos/for_23000.mp4
sam_videos/low_23000.mp4
sam_videos/level_23000.mp4
sam_videos/

In [9]:
from moviepy.editor import VideoFileClip, CompositeVideoClip, vfx
import os

def overlay_videos(base_video_path, overlay_paths, duration=10):
    # Load the base video and cut it to the specified duration
    base_video = VideoFileClip(base_video_path).subclip(0, duration)
    
    # Calculate the size for overlay videos (1/5 of the base video size)
    overlay_width = base_video.w // 5
    overlay_height = base_video.h // 5
    
    # Load all existing overlay videos
    overlay_clips = []
    for path in overlay_paths:
        if os.path.exists(path):
            clip = VideoFileClip(path)
            # Resize the clip to 1/5 of the base video size
            clip = clip.resize(width=overlay_width)
            # Remove the black background
            clip = clip.fx(vfx.mask_color, color=[0, 0, 0], thr=10, s=5)
            overlay_clips.append(clip)
    
    if not overlay_clips:
        print("No overlay videos found.")
        return
    
    # Concatenate all overlay clips
    concatenated_overlay = concatenate_videoclips(overlay_clips, method="compose")
    
    # If the concatenated overlay is shorter than the base video, loop it
    if concatenated_overlay.duration < base_video.duration:
        concatenated_overlay = concatenated_overlay.loop(duration=base_video.duration)
    else:
        # If it's longer, cut it to match the base video duration
        concatenated_overlay = concatenated_overlay.subclip(0, base_video.duration)
    
    # Set the position to bottom right
    concatenated_overlay = concatenated_overlay.set_position(("right", "bottom"))
    
    # Overlay the concatenated clip on the base video
    final_clip = CompositeVideoClip([base_video, concatenated_overlay])
    
    # Generate output path
    output_path = f"{os.path.splitext(base_video_path)[0]}_combined.mp4"
    
    # Write the result to a file
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    
    # Close the clips
    base_video.close()
    for clip in overlay_clips:
        clip.close()
    concatenated_overlay.close()
    final_clip.close()

# Usage
base_video_path = 'S0P3hjM0DDM.mp4'
overlay_paths = generate_video_paths('combined_data.json')  # Your function from the previous answer

overlay_videos(base_video_path, overlay_paths)

chunk:  41%|████████▌            | 408/1001 [09:00<00:00, 1029.43it/s, now=None]

Moviepy - Building video S0P3hjM0DDM_combined.mp4.
MoviePy - Writing audio in S0P3hjM0DDM_combinedTEMP_MPY_wvf_snd.mp4



chunk:  41%|████████▌            | 408/1001 [09:00<00:00, 1029.43it/s, now=None]

MoviePy - Done.
Moviepy - Writing video S0P3hjM0DDM_combined.mp4




chunk:  41%|████████▌            | 408/1001 [09:10<00:00, 1029.43it/s, now=None]

Moviepy - Done !
Moviepy - video ready S0P3hjM0DDM_combined.mp4


In [23]:
from moviepy.editor import VideoFileClip, CompositeVideoClip, vfx
import os
import time
import math

def overlay_videos(base_video_path, overlay_paths, duration=None):
    total_start_time = time.time()

    # Load the base video
    base_video = VideoFileClip(base_video_path)
    
    # If duration is not specified, use the full length of the base video
    if duration is None:
        duration = base_video.duration
    else:
        base_video = base_video.subclip(0, duration)
    
    # Calculate the size for overlay videos (1/4 of the base video size)
    overlay_width = base_video.w // 4
    
    processing_start_time = time.time()
    
    # Check durations of overlay videos and filter out any that don't exist
    existing_overlay_paths = []
    overlay_durations = []
    for path in overlay_paths:
        if os.path.exists(path):
            clip = VideoFileClip(path)
            existing_overlay_paths.append(path)
            overlay_durations.append(clip.duration)
            clip.close()
    
    # Calculate the number of overlay videos to use
    num_overlays = len(existing_overlay_paths)
    
    if num_overlays == 0:
        print("No overlay videos found.")
        return
    
    # Calculate the duration for each overlay clip
    clip_duration = duration / num_overlays
    
    # Load and process all existing overlay videos
    overlay_clips = []
    for i, path in enumerate(existing_overlay_paths):
        clip = VideoFileClip(path)
        
        # Resize the clip to 1/4 of the base video size
        clip = clip.resize(width=overlay_width)
        
        # Remove the black background
        clip = clip.fx(vfx.mask_color, color=[0, 0, 0], thr=10, s=5)
        
        # Calculate the speed factor to slow down the video to fit the clip duration
        speed_factor = overlay_durations[i] / clip_duration
        
        # Slow down the clip to fit the calculated duration
        if speed_factor < 1:
            clip = clip.speedx(factor=speed_factor)
        
        # If the slowed-down clip is still shorter than clip_duration, loop it
        if clip.duration < clip_duration:
            loop_times = math.ceil(clip_duration / clip.duration)
            clip = clip.loop(n=loop_times)
        
        # Set the final duration
        clip = clip.set_duration(clip_duration)
        
        # Set the start time for this clip
        start_time = i * clip_duration
        clip = clip.set_start(start_time)
        
        # Set position (bottom right)
        clip = clip.set_position(("right", "bottom"))
        
        overlay_clips.append(clip)
    
    # Create a composite of all overlay clips
    overlay_composite = CompositeVideoClip(overlay_clips, size=base_video.size)
    
    # Overlay the composite clip on the base video
    final_clip = CompositeVideoClip([base_video, overlay_composite])
    
    processing_end_time = time.time()
    processing_duration = processing_end_time - processing_start_time
    
    # Create 'final_videos' directory if it doesn't exist
    os.makedirs('final_videos', exist_ok=True)
    
    # Generate output path with base video name and duration
    base_name = os.path.splitext(os.path.basename(base_video_path))[0]
    output_path = f"final_videos/{base_name}_duration_{int(duration)}s_combined.mp4"
    
    # Write the result to a file
    writing_start_time = time.time()
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    writing_end_time = time.time()
    writing_duration = writing_end_time - writing_start_time
    
    # Close the clips
    base_video.close()
    for clip in overlay_clips:
        clip.close()
    overlay_composite.close()
    final_clip.close()

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    print(f"Video processing took: {processing_duration:.2f} seconds")
    print(f"Video writing took: {writing_duration:.2f} seconds")
    print(f"Total video generation took: {total_duration:.2f} seconds")
    print(f"Final video saved as: {output_path}")

# Usage
base_video_path = 'S0P3hjM0DDM.mp4'  # Make sure to provide the correct path
overlay_paths = generate_video_paths('combined_data.json')  # Your function from the previous answer
user_duration = 40  # Set your desired duration here, or None to use full video length
overlay_videos(base_video_path, overlay_paths, duration=user_duration)

Moviepy - Building video final_videos/S0P3hjM0DDM_duration_40s_combined.mp4.
MoviePy - Writing audio in S0P3hjM0DDM_duration_40s_combinedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_videos/S0P3hjM0DDM_duration_40s_combined.mp4



Moviepy - Done !
Moviepy - video ready final_videos/S0P3hjM0DDM_duration_40s_combined.mp4
Video processing took: 6.80 seconds
Video writing took: 88.13 seconds
Total video generation took: 95.65 seconds
Final video saved as: final_videos/S0P3hjM0DDM_duration_40s_combined.mp4


In [30]:
from moviepy.editor import VideoFileClip, CompositeVideoClip, vfx
import os
import time
import math

def overlay_videos(base_video_path, overlay_paths, duration=None):
    total_start_time = time.time()

    # Load the base video
    base_video = VideoFileClip(base_video_path)
    
    # If duration is not specified, use the full length of the base video
    if duration is None:
        duration = base_video.duration
    else:
        duration = min(duration, base_video.duration)
    
    # Calculate the fraction of the base video that will be used
    fraction_of_base = duration / base_video.duration
    
    # Calculate the size for overlay videos (1/4 of the base video size)
    overlay_width = base_video.w // 4
    
    processing_start_time = time.time()
    
    # Check durations of overlay videos and filter out any that don't exist
    existing_overlay_paths = []
    overlay_durations = []
    for path in overlay_paths:
        if os.path.exists(path):
            clip = VideoFileClip(path)
            existing_overlay_paths.append(path)
            overlay_durations.append(clip.duration)
            clip.close()
    
    # Calculate the number of overlay videos to use based on the fraction of the base video
    total_available_overlays = len(existing_overlay_paths)
    num_overlays = max(1, min(math.ceil(total_available_overlays * fraction_of_base), total_available_overlays))
    
    if num_overlays == 0:
        print("No overlay videos found.")
        return
    
    # Select a subset of overlay videos
    selected_overlay_paths = existing_overlay_paths[:num_overlays]
    selected_overlay_durations = overlay_durations[:num_overlays]
    
    # Calculate the duration for each overlay clip
    clip_duration = base_video.duration / num_overlays
    
    # Load and process selected overlay videos
    overlay_clips = []
    for i, (path, original_duration) in enumerate(zip(selected_overlay_paths, selected_overlay_durations)):
        clip = VideoFileClip(path)
        
        # Resize the clip to 1/4 of the base video size
        clip = clip.resize(width=overlay_width)
        
        # Remove the black background
        clip = clip.fx(vfx.mask_color, color=[0, 0, 0], thr=10, s=5)
        
        # Calculate the speed factor to adjust the video to fit the clip duration
        speed_factor = original_duration / clip_duration
        
        # Speed up or slow down the clip to fit the calculated duration
        clip = clip.speedx(factor=speed_factor)
        
        # Set the start time for this clip
        start_time = i * clip_duration
        clip = clip.set_start(start_time)
        
        # Set position (bottom right)
        clip = clip.set_position(("right", "bottom"))
        
        overlay_clips.append(clip)
    
    # Create a composite of all overlay clips
    overlay_composite = CompositeVideoClip(overlay_clips, size=base_video.size)
    
    # Overlay the composite clip on the base video
    final_clip = CompositeVideoClip([base_video, overlay_composite]).subclip(0, duration)
    
    processing_end_time = time.time()
    processing_duration = processing_end_time - processing_start_time
    
    # Create 'final_videos' directory if it doesn't exist
    os.makedirs('final_videos', exist_ok=True)
    
    # Generate output path with base video name and duration
    base_name = os.path.splitext(os.path.basename(base_video_path))[0]
    output_path = f"final_videos/{base_name}_duration_{int(duration)}s_combined.mp4"
    
    # Write the result to a file
    writing_start_time = time.time()
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    writing_end_time = time.time()
    writing_duration = writing_end_time - writing_start_time
    
    # Close the clips
    base_video.close()
    for clip in overlay_clips:
        clip.close()
    overlay_composite.close()
    final_clip.close()

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    print(f"Video processing took: {processing_duration:.2f} seconds")
    print(f"Video writing took: {writing_duration:.2f} seconds")
    print(f"Total video generation took: {total_duration:.2f} seconds")
    print(f"Final video saved as: {output_path}")
    print(f"Number of overlay videos used: {num_overlays}")
    print(f"Fraction of base video used: {fraction_of_base:.2f}")

# Usage
base_video_path = 'S0P3hjM0DDM.mp4'  # Make sure to provide the correct path
overlay_paths = generate_video_paths('combined_data.json')  # Your function from the previous answer
user_duration = None  # Set your desired duration here, or None to use full video length
overlay_videos(base_video_path, overlay_paths, duration=user_duration)

t:  61%|█████████████████           | 219/360 [00:53<00:09, 14.37it/s, now=None]

Moviepy - Building video final_videos/S0P3hjM0DDM_duration_45s_combined.mp4.
MoviePy - Writing audio in S0P3hjM0DDM_duration_45s_combinedTEMP_MPY_wvf_snd.mp4



t:  61%|█████████████████           | 219/360 [00:54<00:09, 14.37it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_videos/S0P3hjM0DDM_duration_45s_combined.mp4




t:  61%|█████████████████           | 219/360 [03:26<00:09, 14.37it/s, now=None]

Moviepy - Done !
Moviepy - video ready final_videos/S0P3hjM0DDM_duration_45s_combined.mp4
Video processing took: 7.41 seconds
Video writing took: 153.07 seconds
Total video generation took: 160.88 seconds
Final video saved as: final_videos/S0P3hjM0DDM_duration_45s_combined.mp4
Number of overlay videos used: 38
Fraction of base video used: 1.00


In [28]:
len(overlay_paths)

418